In [ ]:
# App_Rose_Diagram_Jupyter.ipynb (versión Jupyter con ipywidgets)

import pandas as pd
import matplotlib.pyplot as plt
from windrose import WindroseAxes
#import itertools
import ipywidgets as widgets
#from IPython.display import display, clear_output

In [ ]:
# --- Cargar datos ---
RUTA_CSV = '/workspaces/ml-bootcamp-labs/data/raw/DATOS_DIAGRAMA_ROSA_UNIFICADO.csv'
df = pd.read_csv(RUTA_CSV, sep=';')

In [ ]:
# --- Rangos por pozo ---
rangos_pozo = (
    df.groupby('wellName', as_index=False)['TDEP-ft']
      .agg(prof_min='min', prof_max='max')
      .sort_values('wellName')
)
print("📊 Rangos por pozo (ft):")
display(rangos_pozo)

if 'Type' in df.columns:
    rangos_pozo_tipo = (
        df.groupby(['wellName', 'Type'], as_index=False)['TDEP-ft']
          .agg(prof_min='min', prof_max='max')
          .sort_values(['wellName', 'Type'])
    )
    print("📊 Rangos por pozo y tipo (ft):")
    display(rangos_pozo_tipo)

In [ ]:
# --- Widgets de selección ---
pozos = sorted(df['wellName'].dropna().unique())
pozo_sel = widgets.Dropdown(options=pozos, description="Pozo:")

def update_tipo_options(*args):
    tipos = sorted(df.loc[df['wellName'] == pozo_sel.value, 'Type'].dropna().unique())
    tipo_sel.options = tipos

tipo_sel = widgets.Dropdown(description="Tipo:")
pozo_sel.observe(update_tipo_options, names='value')
update_tipo_options()

In [ ]:
# Entradas de profundidad (valores por defecto luego de elegir pozo+tipo)
prof_min_input = widgets.FloatText(description="Prof Min (ft):")
prof_max_input = widgets.FloatText(description="Prof Max (ft):")

def update_prof_range(*args):
    df_tmp = df[(df['wellName'] == pozo_sel.value) & (df['Type'] == tipo_sel.value)]
    if not df_tmp.empty:
        prof_min = float(df_tmp['TDEP-ft'].min())
        prof_max = float(df_tmp['TDEP-ft'].max())
        prof_min_input.value = prof_min
        prof_max_input.value = prof_max

tipo_sel.observe(update_prof_range, names='value')
pozo_sel.observe(update_prof_range, names='value')
update_prof_range()

In [ ]:
# Número de bins
bins_slider = widgets.IntSlider(value=8, min=4, max=36, description="Bins:")

In [ ]:
# Paletas de colores
paletas = {
    'Clásico Geológico': ['#003f5c', '#58508d', '#bc5090', '#ff6361', '#ffa600'],
    'Elegante': ['#264653', '#2a9d8f', '#e9c46a', '#f4a261', '#e76f51'],
    'Marino/Terrestre': ['#00429d', '#4771b2', '#73a2c6', '#a5d5d8', '#f6f5f5']
}
paleta_sel = widgets.Dropdown(options=list(paletas.keys()), value='Clásico Geológico', description="Paleta:")


In [ ]:
# Modo
modo_sel = widgets.RadioButtons(
    options=["Orientaciones puras (solo azimut)", "Orientaciones + clasificación por dip"],
    description="Modo:"
)

In [ ]:
# Botón para generar
btn = widgets.Button(description="Generar diagrama", button_style="success")

In [ ]:
# --- Función para graficar ---
def generar_diagrama(b):
    clear_output(wait=True)
    display(ui)  # vuelve a mostrar UI
    
    df_tmp = df[(df['wellName'] == pozo_sel.value) & (df['Type'] == tipo_sel.value)]
    if df_tmp.empty:
        print("⚠️ No hay datos para esta selección.")
        return

    # Filtrado por profundidades
    df_f = df_tmp[
        (df_tmp['TDEP-ft'] >= prof_min_input.value) &
        (df_tmp['TDEP-ft'] <= prof_max_input.value)
    ]

    if df_f.empty:
        print("⚠️ No hay datos en el rango de profundidad.")
        return

In [ ]:
  # Preparar colores
    colores = paletas[paleta_sel.value]
    colores_final = list(itertools.islice(itertools.cycle(colores), bins_slider.value))

In [ ]:
    # Gráfico
    fig = plt.figure(figsize=(7, 7))
    ax = WindroseAxes.from_ax(fig=fig)

    if modo_sel.value == "Orientaciones puras (solo azimut)":
        ax.bar(
            df_f['Azimuth-dega'],
            [1]*len(df_f),
            bins=bins_slider.value,
            normed=True,
            opening=0.8,
            edgecolor='black',
            colors=colores_final
        )
        ax.set_legend(title="Frecuencia (%)", loc='lower right', bbox_to_anchor=(1.2, 0.1))
        plt.title(f"Diagrama de Rosa (Orientaciones)\n{pozo_sel.value} - {tipo_sel.value}\n{prof_min_input.value:.2f}–{prof_max_input.value:.2f} ft")

    else:
        ax.bar(
            df_f['Azimuth-dega'],
            df_f['Dip_dega'],
            bins=bins_slider.value,
            normed=True,
            opening=0.8,
            edgecolor='black',
            colors=colores_final
        )
        ax.set_legend(title="Frecuencia (%)", loc='lower right', bbox_to_anchor=(1.2, 0.1))
        plt.title(f"Diagrama de Rosa (Orientaciones + Dip)\n{pozo_sel.value} - {tipo_sel.value}\n{prof_min_input.value:.2f}–{prof_max_input.value:.2f} ft")

    plt.show()

btn.on_click(generar_diagrama)

In [ ]:
# --- Interfaz ---
ui = widgets.VBox([
    pozo_sel, tipo_sel,
    prof_min_input, prof_max_input,
    bins_slider, paleta_sel, modo_sel,
    btn
])
display(ui)